In [ ]:
from libs.modeling.attention import Attention_Temporal
from modules.blocks import DropPath,Mlp 
from libs.modeling.mstcn2 import Prediction_Generation

In [ ]:
models_dir= '/home/ubuntu/Dropbox/Rezowan_codebase/dgx_code/output/model_out/TCN'
results_dir= '/home/ubuntu/Dropbox/Rezowan_codebase/dgx_code/output/result/mymodel/myasformer'

In [56]:

import torch
import sys
import os
from models import mymodel
# import models.my_asrf_asformer
import torch.nn.functional as F


from datasets.rarp import create_dataframes, RARPDataset, collate_fn

# create_dataframes,RARPDataset,collate_fn
import random
from libs.modeling import asformer
from torch.utils.data import DataLoader
from torchvision.transforms import Compose
from models import asrf
from utils.train_utils import train,train_ef,validate,evaluate,get_optimizer,get_class_weight,resume,save_checkpoint,get_class_weight_u,EarlyStopping
from train.config import Config
from losses.focal_tmse import ActionSegmentationLoss,BoundaryRegressionLoss
config = Config()

root_folder='.'
sample_rate = 2
batch_size= 1

actions_dict={'0':0, '1':1, '2':2, '3':3, '4':4, '5':5, '6':6, '7':7}
num_classes= len(actions_dict)
video_filename ='video_left.avi'
feature_filename ='feat_2048.npy'
annot_filename ='action_continuous.txt'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
base_test_dir= '/home/ubuntu/Dropbox/Datasets/RARP_datasets/rarp_test_u'
test_dataframe = create_dataframes(base_test_dir,video_filename,feature_filename,annot_filename)

test_data = RARPDataset(
        test_dataframe[:2],
        root_folder,
        num_classes,
        actions_dict,
        sample_rate=sample_rate,
    )

val_loader = DataLoader(
        test_data,
        batch_size=batch_size,
        shuffle=True,
        num_workers=config.num_workers,
        drop_last=True if batch_size > 1 else False,
        collate_fn=collate_fn,
        pin_memory=True
    )
channel_masking_rate=0.3


In [58]:
epoch=90
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = asformer.MyTransformer(3,4,2,2,config.n_features,config.in_channel,num_classes,0.3)
print(model.state_dict().keys())

# predict(models_dir, results_dir, val_loader, epoch, actions_dict, sample_rate,device)

odict_keys(['encoder.PG.conv_1x1_in.weight', 'encoder.PG.conv_1x1_in.bias', 'encoder.PG.conv_dilated_1.0.weight', 'encoder.PG.conv_dilated_1.0.bias', 'encoder.PG.conv_dilated_1.1.weight', 'encoder.PG.conv_dilated_1.1.bias', 'encoder.PG.conv_dilated_1.2.weight', 'encoder.PG.conv_dilated_1.2.bias', 'encoder.PG.conv_dilated_1.3.weight', 'encoder.PG.conv_dilated_1.3.bias', 'encoder.PG.conv_dilated_2.0.weight', 'encoder.PG.conv_dilated_2.0.bias', 'encoder.PG.conv_dilated_2.1.weight', 'encoder.PG.conv_dilated_2.1.bias', 'encoder.PG.conv_dilated_2.2.weight', 'encoder.PG.conv_dilated_2.2.bias', 'encoder.PG.conv_dilated_2.3.weight', 'encoder.PG.conv_dilated_2.3.bias', 'encoder.PG.conv_fusion.0.weight', 'encoder.PG.conv_fusion.0.bias', 'encoder.PG.conv_fusion.1.weight', 'encoder.PG.conv_fusion.1.bias', 'encoder.PG.conv_fusion.2.weight', 'encoder.PG.conv_fusion.2.bias', 'encoder.PG.conv_fusion.3.weight', 'encoder.PG.conv_fusion.3.bias', 'encoder.PG.conv_out.weight', 'encoder.PG.conv_out.bias', 'e

In [104]:
from libs.modeling.mstcn2 import Prediction_Generation
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


def exponential_descrease(idx_decoder, p=3):
    return math.exp(-p * idx_decoder)


class AttentionHelper(nn.Module):
    def __init__(self):
        super(AttentionHelper, self).__init__()
        self.softmax = nn.Softmax(dim=-1)

    def scalar_dot_att(self, proj_query, proj_key, proj_val, padding_mask):
        '''
        scalar dot attention.
        :param proj_query: shape of (B, C, L) => (Batch_Size, Feature_Dimension, Length)
        :param proj_key: shape of (B, C, L)
        :param proj_val: shape of (B, C, L)
        :param padding_mask: shape of (B, C, L)
        :return: attention value of shape (B, C, L)
        '''
        m, c1, l1 = proj_query.shape
        m, c2, l2 = proj_key.shape

        assert c1 == c2

        energy = torch.bmm(proj_query.permute(0, 2, 1), proj_key)  # out of shape (B, L1, L2)
        attention = energy / np.sqrt(c1)
        attention = attention + torch.log(padding_mask + 1e-6)  # mask the zero paddings. log(1e-6) for zero paddings
        attention = self.softmax(attention)
        attention = attention * padding_mask
        attention = attention.permute(0, 2, 1)
        out = torch.bmm(proj_val, attention)
        return out, attention


class AttLayer(nn.Module):
    def __init__(self, q_dim, k_dim, v_dim, r1, r2, r3, bl, stage, att_type):  # r1 = r2
        super(AttLayer, self).__init__()

        self.query_conv = nn.Conv1d(in_channels=q_dim, out_channels=q_dim // r1, kernel_size=1)
        self.key_conv = nn.Conv1d(in_channels=k_dim, out_channels=k_dim // r2, kernel_size=1)
        self.value_conv = nn.Conv1d(in_channels=v_dim, out_channels=v_dim // r3, kernel_size=1)

        self.conv_out = nn.Conv1d(in_channels=v_dim // r3, out_channels=v_dim, kernel_size=1)

        self.bl = bl
        self.stage = stage
        self.att_type = att_type
        assert self.att_type in ['normal_att', 'block_att', 'sliding_att']
        assert self.stage in ['encoder', 'decoder']

        self.att_helper = AttentionHelper()
        self.window_mask = self.construct_window_mask()

    def construct_window_mask(self):
        '''
            construct window mask of shape (1, l, l + l//2 + l//2), used for sliding window self attention
        '''
        window_mask = torch.zeros((1, self.bl, self.bl + 2 * (self.bl // 2)))
        for i in range(self.bl):
            # window_mask[:, :, i:i+self.bl] = 1
            window_mask[:, i, i:i + self.bl] = 1
        return window_mask.to(device)

    def forward(self, x1, x2, mask):
        # x1 from the encoder
        # x2 from the decoder

        query = self.query_conv(x1)
        key = self.key_conv(x1)

        if self.stage == 'decoder':
            assert x2 is not None
            value = self.value_conv(x2)
        else:
            value = self.value_conv(x1)

        if self.att_type == 'normal_att':
            return self._normal_self_att(query, key, value, mask)
        elif self.att_type == 'block_att':
            return self._block_wise_self_att(query, key, value, mask)
        elif self.att_type == 'sliding_att':
            return self._sliding_window_self_att(query, key, value, mask)

    def _normal_self_att(self, q, k, v, mask):
        m_batchsize, c1, L = q.size()
        _, c2, L = k.size()
        _, c3, L = v.size()
        padding_mask = torch.ones((m_batchsize, 1, L)).to(device) * mask[:, 0:1, :]
        output, attentions = self.att_helper.scalar_dot_att(q, k, v, padding_mask)
        output = self.conv_out(F.relu(output))
        output = output[:, :, 0:L]
        return output * mask[:, 0:1, :]

    def _block_wise_self_att(self, q, k, v, mask):
        m_batchsize, c1, L = q.size()
        _, c2, L = k.size()
        _, c3, L = v.size()

        nb = L // self.bl
        if L % self.bl != 0:
            q = torch.cat([q, torch.zeros((m_batchsize, c1, self.bl - L % self.bl)).to(device)], dim=-1)
            k = torch.cat([k, torch.zeros((m_batchsize, c2, self.bl - L % self.bl)).to(device)], dim=-1)
            v = torch.cat([v, torch.zeros((m_batchsize, c3, self.bl - L % self.bl)).to(device)], dim=-1)
            nb += 1

        padding_mask = torch.cat([torch.ones((m_batchsize, 1, L)).to(device) * mask[:, 0:1, :],
                                  torch.zeros((m_batchsize, 1, self.bl * nb - L)).to(device)], dim=-1)

        q = q.reshape(m_batchsize, c1, nb, self.bl).permute(0, 2, 1, 3).reshape(m_batchsize * nb, c1, self.bl)
        padding_mask = padding_mask.reshape(m_batchsize, 1, nb, self.bl).permute(0, 2, 1, 3).reshape(m_batchsize * nb,
                                                                                                     1, self.bl)
        k = k.reshape(m_batchsize, c2, nb, self.bl).permute(0, 2, 1, 3).reshape(m_batchsize * nb, c2, self.bl)
        v = v.reshape(m_batchsize, c3, nb, self.bl).permute(0, 2, 1, 3).reshape(m_batchsize * nb, c3, self.bl)

        output, attentions = self.att_helper.scalar_dot_att(q, k, v, padding_mask)
        output = self.conv_out(F.relu(output))

        output = output.reshape(m_batchsize, nb, c3, self.bl).permute(0, 2, 1, 3).reshape(m_batchsize, c3, nb * self.bl)
        output = output[:, :, 0:L]
        return output * mask[:, 0:1, :]

    def _sliding_window_self_att(self, q, k, v, mask):
        m_batchsize, c1, L = q.size()
        _, c2, _ = k.size()
        _, c3, _ = v.size()

        assert m_batchsize == 1  # currently, we only accept input with batch size 1
        # padding zeros for the last segment
        nb = L // self.bl
        if L % self.bl != 0:
            q = torch.cat([q, torch.zeros((m_batchsize, c1, self.bl - L % self.bl)).to(device)], dim=-1)
            k = torch.cat([k, torch.zeros((m_batchsize, c2, self.bl - L % self.bl)).to(device)], dim=-1)
            v = torch.cat([v, torch.zeros((m_batchsize, c3, self.bl - L % self.bl)).to(device)], dim=-1)
            nb += 1
        padding_mask = torch.cat([torch.ones((m_batchsize, 1, L)).to(device) * mask[:, 0:1, :].to(device),
                                  torch.zeros((m_batchsize, 1, self.bl * nb - L)).to(device)], dim=-1)

        # sliding window approach, by splitting query_proj and key_proj into shape (c1, l) x (c1, 2l)
        # sliding window for query_proj: reshape
        q = q.reshape(m_batchsize, c1, nb, self.bl).permute(0, 2, 1, 3).reshape(m_batchsize * nb, c1, self.bl)

        # sliding window approach for key_proj
        # 1. add paddings at the start and end
        k = torch.cat([torch.zeros(m_batchsize, c2, self.bl // 2).to(device), k,
                       torch.zeros(m_batchsize, c2, self.bl // 2).to(device)], dim=-1)
        v = torch.cat([torch.zeros(m_batchsize, c3, self.bl // 2).to(device), v,
                       torch.zeros(m_batchsize, c3, self.bl // 2).to(device)], dim=-1)
        padding_mask = torch.cat([torch.zeros(m_batchsize, 1, self.bl // 2).to(device), padding_mask,
                                  torch.zeros(m_batchsize, 1, self.bl // 2).to(device)], dim=-1)

        # 2. reshape key_proj of shape (m_batchsize*nb, c1, 2*self.bl)
        k = torch.cat([k[:, :, i * self.bl:(i + 1) * self.bl + (self.bl // 2) * 2] for i in range(nb)],
                      dim=0)  # special case when self.bl = 1
        v = torch.cat([v[:, :, i * self.bl:(i + 1) * self.bl + (self.bl // 2) * 2] for i in range(nb)], dim=0)
        # 3. construct window mask of shape (1, l, 2l), and use it to generate final mask
        padding_mask = torch.cat(
            [padding_mask[:, :, i * self.bl:(i + 1) * self.bl + (self.bl // 2) * 2] for i in range(nb)],
            dim=0)  # of shape (m*nb, 1, 2l)
        final_mask = self.window_mask.repeat(m_batchsize * nb, 1, 1) * padding_mask

        output, attention = self.att_helper.scalar_dot_att(q, k, v, final_mask)
        output = self.conv_out(F.relu(output))

        output = output.reshape(m_batchsize, nb, -1, self.bl).permute(0, 2, 1, 3).reshape(m_batchsize, -1, nb * self.bl)
        output = output[:, :, 0:L]
        return output * mask[:, 0:1, :].to(device)


class MultiHeadAttLayer(nn.Module):
    def __init__(self, q_dim, k_dim, v_dim, r1, r2, r3, bl, stage, att_type, num_head):
        super(MultiHeadAttLayer, self).__init__()
        #         assert v_dim % num_head == 0
        self.conv_out = nn.Conv1d(v_dim * num_head, v_dim, 1)
        self.layers = nn.ModuleList(
            [copy.deepcopy(AttLayer(q_dim, k_dim, v_dim, r1, r2, r3, bl, stage, att_type)) for i in range(num_head)])

        self.dropout = nn.Dropout(p=0.5)

    def forward(self, x1, x2, mask):
        out = torch.cat([layer(x1, x2, mask) for layer in self.layers], dim=1)

        out = self.conv_out(self.dropout(out))
        return out

class ConvFeedForward(nn.Module):
    def __init__(self, dilation, in_channels, out_channels):
        super(ConvFeedForward, self).__init__()
        self.layer = nn.Sequential(
            nn.Conv1d(in_channels, out_channels, 3, padding=dilation, dilation=dilation),
            nn.ReLU()
        )

    def forward(self, x):
        return self.layer(x)

class FCFeedForward(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(FCFeedForward, self).__init__()
        self.layer = nn.Sequential(
            nn.Conv1d(in_channels, out_channels, 1),  # conv1d equals fc
            nn.ReLU(),
            nn.Dropout(),
            nn.Conv1d(out_channels, out_channels, 1)
        )

    def forward(self, x):
        return self.layer(x)

class AttModule(nn.Module):
    def __init__(self, dilation, in_channels, out_channels, r1, r2, att_type, stage, alpha,num_head):
        super(AttModule, self).__init__()
        self.feed_forward = ConvFeedForward(dilation, in_channels, out_channels)
        self.instance_norm = nn.InstanceNorm1d(in_channels, track_running_stats=False)
        # self.att_layer = AttLayer(in_channels, in_channels, out_channels, r1, r1, r2, dilation, att_type=att_type,
        #                           stage=stage)  # dilation
        self.att_layer = MultiHeadAttLayer(in_channels, in_channels, out_channels, r1, r1, r2, dilation,stage=stage, att_type=att_type,num_head=num_head)  # dilation
        self.conv_1x1 = nn.Conv1d(out_channels, out_channels, 1)
        self.dropout = nn.Dropout()
        self.alpha = alpha

    def forward(self, x, f, mask):
        out = self.feed_forward(x)
        out = self.alpha * self.att_layer(self.instance_norm(out), f, mask) + out
        out = self.conv_1x1(out)
        out = self.dropout(out)
        return (x + out) * mask[:, 0:1, :].to(device)


class Encoder(nn.Module):
    def __init__(self, num_layers, r1, r2, num_f_maps, input_dim, num_classes, channel_masking_rate, att_type, alpha,num_head):
        super(Encoder, self).__init__()
        # self.conv_1x1 =  #nn.Conv1d(input_dim, num_f_maps, 1)  # fc layer
        self.PG = Prediction_Generation(num_layers=num_layers, num_f_maps=num_f_maps, dim=input_dim,
                                        num_classes=num_classes)
        self.layers = nn.ModuleList(
            [AttModule(2 ** i, num_f_maps, num_f_maps, r1, r2, att_type, 'encoder', alpha,num_head) for i in  # 2**i
             range(num_layers)])

        self.conv_out = nn.Conv1d(num_f_maps, num_classes, 1)

        self.dropout = nn.Dropout2d(p=channel_masking_rate)
        self.channel_masking_rate = channel_masking_rate

    def forward(self, x, mask):
        '''
        :param x: (N, C, L)
        :param mask:
        :return:
        '''

        if self.channel_masking_rate > 0:
            x = x.unsqueeze(2)
            x = self.dropout(x)
            x = x.squeeze(2)

        o , feature = self.PG(x)
        for layer in self.layers:
            feature = layer(feature, None, mask)

        out = self.conv_out(feature) * mask[:, 0:1, :].to(device)

        return out, feature


class Decoder(nn.Module):
    def __init__(self, num_layers, r1, r2, num_f_maps, input_dim, num_classes, att_type, alpha,num_head,max_len=40000):
        super(Decoder, self).__init__()  # self.position_en = PositionalEncoding(d_model=num_f_maps)
        self.conv_1x1 = nn.Conv1d(input_dim, num_f_maps, 1)
        self.layers = nn.ModuleList(
            [AttModule(2 ** i, num_f_maps, num_f_maps, r1, r2, att_type, 'decoder', alpha,num_head=num_head) for i in  # 2 ** i
             range(num_layers)])
        self.conv_out = nn.Conv1d(num_f_maps, num_classes, 1)

    def forward(self, x, fencoder, mask):
        feature = self.conv_1x1(x)
        for layer in self.layers:
            feature = layer(feature, fencoder, mask)

        out = self.conv_out(feature) * mask[:, 0:1, :]

        return out, feature


class MyTransformer(nn.Module):
    def __init__(self, num_decoders, num_layers, r1, r2, num_f_maps, input_dim, num_classes, channel_masking_rate,num_head):
        super(MyTransformer, self).__init__()
        self.encoder = Encoder(num_layers, r1, r2, num_f_maps, input_dim, num_classes, channel_masking_rate,
                               att_type='sliding_att', alpha=1,num_head=num_head)
        self.decoders = nn.ModuleList([copy.deepcopy(
            Decoder(num_layers, r1, r2, num_f_maps, num_classes, num_classes, att_type='sliding_att',
                    alpha=exponential_descrease(s),num_head=num_head)) for s in range(num_decoders)])  # num_decoders
        self.conv_bound = nn.Conv1d(num_f_maps, 3, 1)
    def forward(self, x, mask):
        mask=mask.unsqueeze(1).to(device)
        out, feature = self.encoder(x, mask)
        outputs = out.unsqueeze(0)

        for decoder in self.decoders:
            out, feature = decoder(F.softmax(out, dim=1) * mask[:, 0:1, :], feature * mask[:, 0:1, :], mask)
            outputs = torch.cat((outputs, out.unsqueeze(0)), dim=0)
        bounds = self.conv_bound(feature)
        return outputs,bounds


import copy,math
num_layers=4
r1=2;r2=2
num_f_maps=64; input_dim=2048;channel_masking_rate=0.3;
model = MyTransformer(3, num_layers, r1, r2, num_f_maps, input_dim, num_classes, channel_masking_rate,num_head=4)


In [115]:

model = MyTransformer(3, 11, r1, r2, num_f_maps, input_dim, num_classes, channel_masking_rate,num_head=4)

# num_decoders, num_layers, r1, r2, num_f_maps, input_dim, num_classes, channel_masking_rate,num_head,device

# trained_model = torch.load(model_dir + "/epoch-" + str(90) + ".model")
# for key in trained_model.keys():
#     print(key)
# model.load_state_dict(torch.load(model_dir + "/epoch-" + str(epoch) + ".model"))
# model.state_dict()

# Load the state dict
checkpoint = torch.load(model_dir + "/epoch-" + str(epoch) + ".model")
model.load_state_dict(checkpoint, strict=False) 

/tmp/ipykernel_687084/3550284192.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(model_dir + "/epoch-" + str(epoch) + ".model")


_IncompatibleKeys(missing_keys=['encoder.PG.conv_1x1_in.weight', 'encoder.PG.conv_1x1_in.bias', 'encoder.PG.conv_dilated_1.0.weight', 'encoder.PG.conv_dilated_1.0.bias', 'encoder.PG.conv_dilated_1.1.weight', 'encoder.PG.conv_dilated_1.1.bias', 'encoder.PG.conv_dilated_1.2.weight', 'encoder.PG.conv_dilated_1.2.bias', 'encoder.PG.conv_dilated_1.3.weight', 'encoder.PG.conv_dilated_1.3.bias', 'encoder.PG.conv_dilated_1.4.weight', 'encoder.PG.conv_dilated_1.4.bias', 'encoder.PG.conv_dilated_1.5.weight', 'encoder.PG.conv_dilated_1.5.bias', 'encoder.PG.conv_dilated_1.6.weight', 'encoder.PG.conv_dilated_1.6.bias', 'encoder.PG.conv_dilated_1.7.weight', 'encoder.PG.conv_dilated_1.7.bias', 'encoder.PG.conv_dilated_1.8.weight', 'encoder.PG.conv_dilated_1.8.bias', 'encoder.PG.conv_dilated_1.9.weight', 'encoder.PG.conv_dilated_1.9.bias', 'encoder.PG.conv_dilated_1.10.weight', 'encoder.PG.conv_dilated_1.10.bias', 'encoder.PG.conv_dilated_2.0.weight', 'encoder.PG.conv_dilated_2.0.bias', 'encoder.PG.co

In [154]:


def get_labels_start_end_time(frame_wise_labels, bg_class=[0]):
    labels = []
    starts = []
    ends = []
    last_label = frame_wise_labels[0]
    if frame_wise_labels[0] not in bg_class:
        labels.append(frame_wise_labels[0])
        starts.append(0)
    for i in range(len(frame_wise_labels)):
        if frame_wise_labels[i] != last_label:
            if frame_wise_labels[i] not in bg_class:
                labels.append(frame_wise_labels[i])
                starts.append(i)
            if last_label not in bg_class:
                ends.append(i)
            last_label = frame_wise_labels[i]
    if last_label not in bg_class:
        ends.append(i)
    return labels, starts, ends


def levenstein(p, y, norm=False):
    m_row = len(p)
    n_col = len(y)
    D = np.zeros([m_row + 1, n_col + 1], dtype=np.float64)
    for i in range(m_row + 1):
        D[i, 0] = i
    for i in range(n_col + 1):
        D[0, i] = i

    for j in range(1, n_col + 1):
        for i in range(1, m_row + 1):
            if y[j - 1] == p[i - 1]:
                D[i, j] = D[i - 1, j - 1]
            else:
                D[i, j] = min(D[i - 1, j] + 1,
                              D[i, j - 1] + 1,
                              D[i - 1, j - 1] + 1)

    if norm:
        score = (1 - D[-1, -1] / max(m_row, n_col)) * 100
    else:
        score = D[-1, -1]

    return score


def edit_score(recognized, ground_truth, norm=True, bg_class=[0.0]):
    P, _, _ = get_labels_start_end_time(recognized, bg_class)
    Y, _, _ = get_labels_start_end_time(ground_truth, bg_class)
    return levenstein(P, Y, norm)


def f_score(recognized, ground_truth, overlap, bg_class=[0.0]):
    p_label, p_start, p_end = get_labels_start_end_time(recognized, bg_class)
    y_label, y_start, y_end = get_labels_start_end_time(ground_truth, bg_class)

    tp = 0
    fp = 0

    hits = np.zeros(len(y_label))

    for j in range(len(p_label)):
        intersection = np.minimum(p_end[j], y_end) - np.maximum(p_start[j], y_start)
        union = np.maximum(p_end[j], y_end) - np.minimum(p_start[j], y_start)
        IoU = (1.0 * intersection / union) * ([p_label[j] == y_label[x] for x in range(len(y_label))])
        # Get the best scoring segment
        idx = np.array(IoU).argmax()

        if IoU[idx] >= overlap and not hits[idx]:
            tp += 1
            hits[idx] = 1
        else:
            fp += 1
    fn = len(y_label) - sum(hits)
    return float(tp), float(fp), float(fn)


def segment_bars(save_path, *labels):
    num_pics = len(labels)
    color_map = plt.get_cmap('seismic')
    # color_map =
    fig = plt.figure(figsize=(15, num_pics * 1.5))

    barprops = dict(aspect='auto', cmap=color_map,
                    interpolation='nearest', vmin=0, vmax=20)

    for i, label in enumerate(labels):
        plt.subplot(num_pics, 1, i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.imshow([label], **barprops)

    if save_path is not None:
        plt.savefig(save_path)
    else:
        plt.show()

    plt.close()


def segment_bars_with_confidence(save_path, confidence, *labels):
    num_pics = len(labels) + 1
    color_map = plt.get_cmap('seismic')

    axprops = dict(xticks=[], yticks=[], frameon=False)
    barprops = dict(aspect='auto', cmap=color_map,
                    interpolation='nearest', vmin=0)
    fig = plt.figure(figsize=(15, num_pics * 1.5))

    interval = 1 / (num_pics + 1)
    for i, label in enumerate(labels):
        i = i + 1
        ax1 = fig.add_axes([0, 1 - i * interval, 1, interval])
        ax1.imshow([label], **barprops)

    ax4 = fig.add_axes([0, interval, 1, interval])
    ax4.set_xlim(0, len(confidence))
    ax4.set_ylim(0, 1)
    ax4.plot(range(len(confidence)), confidence)
    ax4.plot(range(len(confidence)), [0.3] * len(confidence), color='red', label='0.5')

    if save_path is not None:
        plt.savefig(save_path)
    else:
        plt.show()

    plt.close()


def func_eval(preds,gts):
    overlap = [.1, .25, .5]
    tp, fp, fn = np.zeros(3), np.zeros(3), np.zeros(3)

    correct = 0
    total = 0
    edit = 0

    for pred,gt in zip(preds,gts):

        gt_content = gt

        recog_content = pred 

        for i in range(len(gt_content)):
            total += 1
            if gt_content[i] == recog_content[i]:
                correct += 1

        edit += edit_score(recog_content, gt_content)

        for s in range(len(overlap)):
            tp1, fp1, fn1 = f_score(recog_content, gt_content, overlap[s])
            tp[s] += tp1
            fp[s] += fp1
            fn[s] += fn1

    acc = 100 * float(correct) / total
    edit = (1.0 * edit) / len(preds)
    #     print("Acc: %.4f" % (acc))
    #     print('Edit: %.4f' % (edit))
    f1s = np.array([0, 0, 0], dtype=float)
    for s in range(len(overlap)):

        precision = tp[s] / float(tp[s] + fp[s])
        recall = tp[s] / float(tp[s] + fn[s])

        # Calculate F1 score
        if precision + recall == 0:
            f1 = 0.0  # Handle the case where both precision and recall are zero
        else:
            f1 = 2.0 * (precision * recall) / (precision + recall)
        
        print(f"Precision: {precision}, Recall: {recall}, F1 Score: {f1} - tp {tp[s]}  fp-  {fp[s]}")

        
        # f1 = 2.0 * (precision * recall) / (precision + recall)

        f1 = np.nan_to_num(f1) * 100
        #         print('F1@%0.2f: %.4f' % (overlap[s], f1))
        f1s[s] = f1

    return acc, edit, f1s


In [139]:
import numpy as np 
import torch.nn.functional as F 
def predict( model_dir, results_dir, val_loader, epoch, actions_dict, sample_rate,device):
    all_gt=[]
    all_recognition=[]
    model.eval()
    with torch.no_grad():
        sample_rate
        # model.load_state_dict(torch.load(model_dir + "/epoch-" + str(epoch) + ".model"))
        model.to(device)  # Move model to device
        for sample in val_loader:
            print(sample)
            x = sample["feature"]
            t = sample["label"]
            b = sample["boundary"]
            mask = sample["mask"]

            x = x.to(device)
            t = t.to(device)
            b = b.to(device)
            mask = mask.to(device)

            # compute output and loss
            output_cls, output_bound = model(x,mask)

            # calcualte accuracy and f1 score
            output_cls = output_cls.to("cpu").data.numpy()
            output_bound = output_bound.to("cpu").data.numpy()
            if isinstance(output_cls, np.ndarray):
                output_cls = torch.from_numpy(output_cls).to(device)
            
            # Now apply softmax
            predictions = output_cls
            print('predictions shape ',predictions.shape)
            for i in range(len(predictions)):
                # Ensure that the tensor is on the correct device
                print(predictions)
                confidence, predicted = torch.max(F.softmax(predictions[i], dim=1).data, 1) # or dim=1 depending on your shape
                # confidence, predicted = torch.max(softmaxed_output.data, 0)  # or 1 depending on your dimension
                confidence, predicted = confidence.squeeze(), predicted.squeeze()

                t = t.squeeze()
                confidence, predicted = confidence.squeeze(), predicted.squeeze()

                print('prediction and terget after squeez ',predicted.shape, t.shape)
                t = t.squeeze()
                # confidence, predicted = confidence.squeeze(), predicted.squeeze()
                

                # segment_bars_with_confidence(results_dir + '/{}_stage{}.png'.format(vid, i),
                #                              confidence.tolist(),
                #                              t.tolist(), predicted.tolist())

            recognition = []
            
            for i in range(len(predicted)):
                print('predicted[i].item()', predicted[i].item())
                recognition = np.concatenate((recognition, [list(actions_dict.keys())[
                                                                list(actions_dict.values()).index(
                                                                    predicted[i].item())]] * sample_rate))
            # # Process predictions for recognition
            # for time_step in range(predicted.shape[1]):  # Iterate over time steps
            #     pred = predicted[:, time_step]  # Get all class scores for the current time step
            #     tgt= t[time_step]
            #     print(pred,tgt)
            #     predicted_class = torch.argmax(pred).item()  # Get the index of the max score
                
                # print('Predicted class for time step {}: {} and target {}'.format(time_step, predicted_class, tgt))
                
                # action = actions_dict.get(predicted_class)  # Get action from index
                # if action is not None:  # Ensure action exists
                #     recognition = [action] * sample_rate  # Repeat action for sample_rate
                #     all_recognition.append(recognition)  # Append to recognition list
                            
            # f_name = vid.split('/')[-1].split('.')[0]
            # f_ptr = open(results_dir + "/" + f_name, "w")
            # f_ptr.write("### Frame level recognition: ###\n")
            # f_ptr.write(' '.join(recognition))
            # f_ptr.close()
            all_recognition.append(recognition)
            all_gt.append(t)
        # time_end = time.time()
        
        return all_recognition,all_gt
    
# ground_truth= np.load(label_file)
all_recognition,all_gt= predict(models_dir,results_dir,val_loader,epoch,actions_dict,sample_rate,device)

{'feature': tensor([[[-0.1686, -0.2614, -0.4153,  ...,  0.0180, -0.0881, -0.0016],
         [ 0.2229,  0.1768,  0.1402,  ..., -0.1503, -0.1350, -0.0875],
         [ 0.1079,  0.2362,  0.1933,  ..., -0.0029,  0.0183,  0.0556],
         ...,
         [-0.0251, -0.1172, -0.1618,  ..., -0.1868, -0.2522, -0.2260],
         [ 0.1491,  0.1651,  0.1448,  ..., -0.0462, -0.0796, -0.0300],
         [-0.1653, -0.2657, -0.1982,  ..., -0.5783, -0.6174, -0.5781]]]), 'label': tensor([[0, 0, 0,  ..., 0, 0, 0]]), 'boundary': tensor([[1., 1., 1.,  ..., 2., 2., 2.]]), 'feature_path': ['/home/ubuntu/Dropbox/Datasets/RARP_datasets/rarp_test_u/video_41/feat_2048.npy'], 'mask': tensor([[True, True, True,  ..., True, True, True]])}
predictions shape  torch.Size([4, 1, 8, 7139])
tensor([[[[ 1.7991e+00,  1.3934e+00,  2.5674e+00,  ...,  5.7610e-01,
           -2.4248e-01, -3.5368e-01],
          [-8.7879e-01, -1.8690e+00, -1.3775e+00,  ...,  4.8860e-01,
            8.3375e-01,  7.0118e-01],
          [-5.1739e-01,

In [143]:
for p,t in zip(all_recognition,all_gt):
    print(p ,t )

['1' '1' '5' ... '1' '1' '1'] tensor([0, 0, 0,  ..., 0, 0, 0], device='cuda:0')
['1' '1' '5' ... '1' '1' '1'] tensor([0, 0, 0,  ..., 0, 0, 0], device='cuda:0')


In [155]:
func_eval(all_recognition,all_gt)

Precision: 0.0, Recall: 0.0, F1 Score: 0.0 - tp 0.0  fp-  18.0
Precision: 0.0, Recall: 0.0, F1 Score: 0.0 - tp 0.0  fp-  18.0
Precision: 0.0, Recall: 0.0, F1 Score: 0.0 - tp 0.0  fp-  18.0


(0.0, 0.0, array([0., 0., 0.]))